# Tutorial 5 -- Human Detection 
This tutorial will show you how to detect human with the robot. We will use multi-thread to capture the color images and depth images. Then the ssd mobilenet v2 nerual network structure is used for human detection. To enable real-time performance, the tensorRT deep learning framework is used. More information can be found in the nvidia's jetbot project website: https://github.com/NVIDIA-AI-IOT/jetbot

### Step 1 load the pre-trained object detection model

**Before running the program, we need to download the pre-trained detection model and put it in the current folder.** The model can be found in the following link:https://drive.google.com/file/d/1KjlDMRD8uhgQmQK-nC2CZGHFTbq4qQQH/view

In [1]:

import tensorrt as trt
from tensorrt_model import TRTModel
from ssd_tensorrt import load_plugins, parse_boxes,TRT_INPUT_NAME, TRT_OUTPUT_NAME
import ctypes
import numpy as np
import cv2
import os
import ctypes
   
mean = 255.0 * np.array([0.5, 0.5, 0.5])
stdev = 255.0 * np.array([0.5, 0.5, 0.5])


def bgr8_to_ssd_input(camera_value):
   x = camera_value
   x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
   x = x.transpose((2, 0, 1)).astype(np.float32)
   x -= mean[:, None, None]
   x /= stdev[:, None, None]
   return x[None, ...]


class ObjectDetector(object):
   
   def __init__(self, engine_path, preprocess_fn=bgr8_to_ssd_input):
       logger = trt.Logger()
       trt.init_libnvinfer_plugins(logger, '')
       load_plugins()
       self.trt_model = TRTModel(engine_path, input_names=[TRT_INPUT_NAME],output_names=[TRT_OUTPUT_NAME, TRT_OUTPUT_NAME + '_1'])
       self.preprocess_fn = preprocess_fn
       
   def execute(self, *inputs):
       trt_outputs = self.trt_model(self.preprocess_fn(*inputs))
       return parse_boxes(trt_outputs)
   
   def __call__(self, *inputs):
       return self.execute(*inputs)


model = ObjectDetector('ssd_mobilenet_v2_coco.engine')

### Step 2 Initialize the camera instance for the Intel realsense sensor D435i

In [2]:
import ipywidgets.widgets as widgets
from IPython.display import display, HTML
#use traitlets and widgets to display the image in Jupyter Notebook
import traitlets
from traitlets.config.configurable import SingletonConfigurable

#use opencv to covert the depth image to RGB image for displaying purpose
import cv2
import numpy as np

#using realsense to capture the color and depth image
import pyrealsense2 as rs

#multi-threading is used to capture the image in real time performance
import threading

class Camera(SingletonConfigurable):
    
    #this changing of this value will be captured by traitlets
    color_value = traitlets.Any()
    
    def __init__(self):
        super(Camera, self).__init__()
        
        #configure the color and depth sensor
        self.pipeline = rs.pipeline()
        self.configuration = rs.config()  
        
        #set resolution for the color camera
        self.color_width = 640
        self.color_height = 480
        self.color_fps = 30
        self.configuration.enable_stream(rs.stream.color, self.color_width, self.color_height, rs.format.bgr8, self.color_fps)

        #set resolution for the depth camera
        self.depth_width = 640
        self.depth_height = 480
        self.depth_fps = 30
        self.configuration.enable_stream(rs.stream.depth, self.depth_width, self.depth_height, rs.format.z16, self.depth_fps)

        #flag to control the thread
        self.thread_runnning_flag = False
        
        #start the RGBD sensor
        self.pipeline.start(self.configuration)
        self.pipeline_started = True
        frames = self.pipeline.wait_for_frames()

        #start capture the first color image
        color_frame = frames.get_color_frame()   
        image = np.asanyarray(color_frame.get_data())
        self.color_value = image

        #start capture the first depth image
        depth_frame = frames.get_depth_frame()           
        self.depth_image = np.asanyarray(depth_frame.get_data())
        depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(self.depth_image, alpha=0.03), cv2.COLORMAP_JET)
        self.depth_value = depth_colormap   

    def _capture_frames(self):
        while(self.thread_runnning_flag==True): #continue until the thread_runnning_flag is set to be False
            frames = self.pipeline.wait_for_frames() #receive data from RGBD sensor
            
            color_frame = frames.get_color_frame() #get the color image
            image = np.asanyarray(color_frame.get_data()) #convert color image to numpy array
            self.color_value = image #assign the numpy array image to the color_value variable 

            depth_frame = frames.get_depth_frame() #get the depth image           
            self.depth_image = np.asanyarray(depth_frame.get_data()) #convert depth data to numpy array
            #conver depth data to BGR image for displaying purpose
            depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(self.depth_image, alpha=0.03), cv2.COLORMAP_JET)
            self.depth_value = depth_colormap #assign the color BGR image to the depth value
    
    def start(self): #start the data capture thread
        if self.thread_runnning_flag == False: #only process if no thread is running yet
            self.thread_runnning_flag=True #flag to control the operation of the _capture_frames function
            self.thread = threading.Thread(target=self._capture_frames) #link thread with the function
            self.thread.start() #start the thread

    def stop(self): #stop the data capture thread
        if self.thread_runnning_flag == True:
            self.thread_runnning_flag = False #exit the while loop in the _capture_frames
            self.thread.join() #wait the exiting of the thread 
            
    def getDepth(self,x,y):
        if x > 640:
            x = 620
        if x < 0:
            x = 0
        if y > 480:
            y = 460
        if y < 0:
            y = 0
        return self.depth_image[int(x),int(y)]
            

def bgr8_to_jpeg(value):#convert numpy array to jpeg coded data for displaying 
    return bytes(cv2.imencode('.jpg',value)[1])

#create a camera object
camera = Camera.instance()
camera.start() # start capturing the data


### Step 3 Perform object detection and display the results on widgets
Human is labeled is 1 in the pretrained model. A full list of the detection class indices can be found in the following link https://github.com/tensorflow/models/blob/master/research/object_detection/data/mscoco_complete_label_map.pbtxt

The following program will ask the robot to move forward if a human is detected in the captured color image.

In [3]:
import ipywidgets.widgets as widgets
from IPython.display import display, HTML

width = 640
height = 480

image_widget = widgets.Image(format='jpeg', width=300, height=300)
label_widget = widgets.IntText(value=1, description='tracked label')
display(widgets.VBox([
    widgets.HBox([image_widget,]),
    label_widget
]))

import time
from RobotClass import Robot

#initialize the Robot class
robot = Robot()

def processing(change):
    image = change['new']
     
    imgsized= cv2.resize(image,(300,300))
    # compute all detected objects
    detections = model(imgsized)
    
    matching_detections = [d for d in detections[0] if d['label'] == int(label_widget.value)]
    
    for det in matching_detections:
        bbox = det['bbox']
        cv2.rectangle(image, (int(width * bbox[0]), int(height * bbox[1])), (int(width * bbox[2]), int(height * bbox[3])), (255, 0, 0), 2)
        #text = str(det['label'])
        #org = (int(width * bbox[0]), int(height * bbox[1])), (int(width * bbox[2]), int(height * bbox[3]))
        #font = cv2.FONT_HERSHEY_SIMPLEX
        #fontsize = 1
        #colour = (0,255,0)
        #print(str(det['label']))
        
    if len(matching_detections)==0:
        robot.stop()
    else:
        # get center of box
        
    image_widget.value = bgr8_to_jpeg(image)
    
#the camera.observe function will monitor the color_value variable. If this value changes, the excecute function will be excuted.
camera.observe(processing, names='color_value')

Exception in thread Thread-4:
Traceback (most recent call last):
  File "/usr/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-2-3e725fea98a1>", line 66, in _capture_frames
    self.color_value = image #assign the numpy array image to the color_value variable
  File "/usr/local/lib/python3.6/dist-packages/traitlets/traitlets.py", line 585, in __set__
    self.set(obj, value)
  File "/usr/local/lib/python3.6/dist-packages/traitlets/traitlets.py", line 574, in set
    obj._notify_trait(self.name, old_value, new_value)
  File "/usr/local/lib/python3.6/dist-packages/traitlets/traitlets.py", line 1139, in _notify_trait
    type='change',
  File "/usr/local/lib/python3.6/dist-packages/traitlets/traitlets.py", line 1176, in notify_change
    c(change)
  File "<ipython-input-3-3fadc47b955f>", line 37, in processing
    cv2.putText(image, text,

In [3]:

import ipywidgets.widgets as widgets
from IPython.display import display, HTML


width = 640
height = 480


#image_widget = widgets.Image(format='jpeg', width=300, height=300)
label_widget = widgets.IntText(value=1, description='tracked label')
#display(widgets.VBox([
#   widgets.HBox([image_widget,]),
#   label_widget
#]))

display_color = widgets.Image(format='jpeg', width='45%') #determine the width of the color image
display_depth = widgets.Image(format='jpeg', width='45%')  #determine the width of the depth image
layout=widgets.Layout(width='100%')

sidebyside = widgets.HBox([display_color, display_depth],layout=layout) #horizontal 
display(sidebyside) #display the widget

import time
from RobotClass import Robot


#initialize the Robot class
robot = Robot()


def processing(change):
    global robot
    image = change['new']
   
    # Resize image to match model's expected sizing
    imgsized = cv2.resize(image, (300, 300))
    # Perform object detection
    detections = model(imgsized)
   
    # Filter detections for the human class (class ID 1)
    matching_detections = [d for d in detections[0] if d['label'] == int(label_widget.value)]
   
    if len(matching_detections) > 0:
        # Assume first detection is the target
        det = matching_detections[0]
        bbox = det['bbox']
        # Draw bounding box
        cv2.rectangle(image, (int(width * bbox[0]), int(height * bbox[1])), (int(width * bbox[2]), int(height * bbox[3])), (255, 0, 0), 2)

       # Calculate center of detection
        detected_center_x = (bbox[0] + bbox[2]) / 2.0
        detected_center_y = (bbox[1] + bbox[3]) / 2.0
        print(camera.getDepth(int(detected_center_x * width), int(detected_center_y * height)), "X: ", detected_center_x * width, "Y: ",detected_center_y * height)
        screen_center_x = 0.5

       # Determine direction based on position of detected center vs screen center
        if detected_center_x < screen_center_x - 0.15:  # Adjust tolerance as needed
            robot.left(0.5)  # Turn left
        elif detected_center_x > screen_center_x + 0.15:
            robot.right(0.5)  # Turn right
        else:
            robot.forward(0.3)  # Move forward if roughly centered
    else:
        robot.stop()  # Stop if no human detected
   
   # Update the image widget
    display_color.value = bgr8_to_jpeg(cv2.resize(image,(160,120)))
    display_depth.value = bgr8_to_jpeg(cv2.resize(camera.depth_value,(160,120)))


#the camera.observe function will monitor the color_value variable. If this value changes, the excecute function will be excuted.
camera.observe(processing, names='color_value')

2969 X:  171.2018346786499 Y:  251.20978116989136
3658 X:  156.81435585021973 Y:  251.89398050308228
579 X:  448.58638763427734 Y:  230.97732067108154


Exception in thread Thread-4:
Traceback (most recent call last):
  File "/usr/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-2-48f654acf3cf>", line 66, in _capture_frames
    self.color_value = image #assign the numpy array image to the color_value variable
  File "/usr/local/lib/python3.6/dist-packages/traitlets/traitlets.py", line 585, in __set__
    self.set(obj, value)
  File "/usr/local/lib/python3.6/dist-packages/traitlets/traitlets.py", line 574, in set
    obj._notify_trait(self.name, old_value, new_value)
  File "/usr/local/lib/python3.6/dist-packages/traitlets/traitlets.py", line 1139, in _notify_trait
    type='change',
  File "/usr/local/lib/python3.6/dist-packages/traitlets/traitlets.py", line 1176, in notify_change
    c(change)
  File "<ipython-input-3-f658433fde94>", line 53, in processing
    print(camera.getDepth(in

In [4]:
camera.unobserve_all()
camera.stop()
time.sleep(1.0)
robot.stop()

## Tasks

### 1. Please try to calculate the distance between the detected human and the robot using the depth image. (Note: You can refer to tutorial 2 to get the depth information for a specific point)
### 2. Please try to add collision avoidance function into this program to protect the robot. (Note: You can refer to tutorial 4 for the collision avoidance)
### 3. Think about how to control the robot to move towards the detected human